In [ ]:
import os
import glob
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from ultralytics import YOLO
import mediapipe as mp

class PalmPhysiognomyAnalyzer:
    def __init__(self, model_path):
        # 1. 모델 및 미디어파이프 로드
        self.model = YOLO(model_path)
        self.mp_hands = mp.solutions.hands.Hands(static_image_mode=True, max_num_hands=1)

        # 2. 설정: 임계값 및 라벨
        self.LINE_MAP = {0: 'fate', 1: 'head', 2: 'heart', 3: 'life'}

    def preprocess_image(self, img_path):
        """이미지 로드 및 CLAHE 전처리"""
        img = cv2.imread(img_path)
        if img is None: return None, None

        # CLAHE (대비 향상)
        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        img_clahe = cv2.cvtColor(cv2.merge((clahe.apply(l), a, b)), cv2.COLOR_LAB2BGR)

        return img, img_clahe

    def get_hand_metrics(self, img):
        """미디어파이프로 손 크기 측정 및 구(Mount) 위치 계산"""
        h, w = img.shape[:2]
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = self.mp_hands.process(img_rgb)

        if not results.multi_hand_landmarks:
            return None

        lm = results.multi_hand_landmarks[0].landmark
        def p(idx): return np.array([int(lm[idx].x * w), int(lm[idx].y * h)])

        # 손 크기 기준 (손목 ~ 중지 뿌리)
        hand_scale = np.linalg.norm(p(9) - p(0))

        # 구(Mount) 위치 정의
        mounts = {
            'jupiter': p(5),   # 목성구 (검지)
            'saturn': p(9),    # 토성구 (중지)
            'sun': p(13),      # 태양구 (약지)
            'mercury': p(17),  # 수성구 (소지)
            'moon': p(0) + (p(17)-p(0))*0.8, # 월구
            'venus': p(0) + (p(5)-p(0))*0.8, # 금성구
            'wrist': p(0)      # 손목
        }

        return {'scale': hand_scale, 'mounts': mounts}

    def extract_geometric_features(self, points, hand_scale):
        """기하학적 특징 추출 (길이, 곡률, 기울기)"""
        if len(points) < 2: return None

        # 1. 길이 비율 (Length Ratio)
        arc_len = np.sum(np.sqrt(np.sum(np.diff(points, axis=0)**2, axis=1)))
        len_ratio = arc_len / hand_scale

        # 2. 곡률 (Curvature)
        euclidean = np.linalg.norm(points[-1] - points[0])
        curvature = arc_len / (euclidean + 1e-6)

        # 3. 기울기 (Slope)
        vec = points[-1] - points[0]
        slope = vec[1] / (vec[0] + 1e-6)

        return {'len_ratio': len_ratio, 'curv': curvature, 'slope': slope, 'points': points}

    def analyze_life_line(self, features, mounts, hand_scale):
        """생명선 분석: 수명, 활력"""
        if not features: return {"status": "Not Detected"}

        pts = features['points']
        # 끝점의 Y좌표가 손목(wrist)에 얼마나 가까운지
        end_y = pts[-1][1] if pts[0][1] < pts[-1][1] else pts[0][1]
        wrist_y = mounts['wrist'][1]

        # 길이 판별
        if end_y > wrist_y - (hand_scale * 0.15): length_type = "Long (장수운)"
        elif end_y > wrist_y - (hand_scale * 0.3): length_type = "Middle (건강함)"
        else: length_type = "Short (단기 집중형)"

        # 곡률 판별 (금성구 침범 여부)
        max_x = np.max(pts[:, 0]) # 오른손 기준
        curve_type = "Energetic (에너자이저)" if max_x > mounts['saturn'][0] else "Delicate (섬세함)"

        return {"Length": length_type, "Vitality": curve_type, "Ratio": f"{features['len_ratio']:.2f}"}

    def analyze_head_line(self, features, mounts):
        """두뇌선 분석: 적성, 사고방식"""
        if not features: return {"status": "Not Detected"}

        # 형태 (직선/곡선)
        shape = "Straight (이과형/논리적)" if features['curv'] < 1.06 else "Curved (문과형/창의적)"

        # 길이 (약지 기준선 넘는지)
        pts = features['points']
        end_x = pts[-1][0] if pts[0][0] < pts[-1][0] else pts[0][0] # 왼쪽이 시작점이라 가정 시 끝점
        length = "Deep Thinker (숙고형)" if end_x > mounts['sun'][0] else "Intuitive (직관형)"

        return {"Shape": shape, "Style": length}

    def analyze_heart_line(self, features, mounts):
        """감정선 분석: 애정관"""
        if not features: return {"status": "Not Detected"}

        # 시작점 높이 (소지 아래)
        start_y = features['points'][0][1] # 보통 가장 왼쪽이나 오른쪽 (데이터에 따라 다름)
        # 여기서는 단순화하여 곡률로 성향 판단
        style = "Cool/Straight (돌직구)" if features['curv'] < 1.04 else "Emotional/Curved (감성적)"

        return {"Love Style": style, "Curvature": f"{features['curv']:.2f}"}

    def analyze_fate_line(self, features, mounts):
        """운명선 분석: 인생의 흐름"""
        if not features: return {"Result": "No Fate Line (자유로운 영혼)"}

        # 시작점 위치 (X좌표)로 판별
        pts = features['points']
        start_x = pts[0][0] if pts[0][1] > pts[-1][1] else pts[-1][0] # Y가 큰 쪽이 손목(시작점)

        center_x = mounts['saturn'][0] # 중지(토성구) X좌표가 대략 중앙

        if abs(start_x - center_x) < 50: origin = "Self-made (자수성가)"
        elif start_x > center_x: origin = "Popularity (인기/후원)" # 월구 쪽
        else: origin = "Family Support (가족 지원)" # 금성구 쪽

        return {"Origin": origin, "Strength": "Strong" if len(pts) > 10 else "Weak"}

    def run(self, img_path):
        """전체 파이프라인 실행"""
        # 1. 이미지 로드
        img, ai_input = self.preprocess_image(img_path)
        if img is None: return None

        # 2. 손 구조 분석 (MediaPipe)
        metrics = self.get_hand_metrics(img)
        if not metrics:
            print("❌ 손을 찾을 수 없습니다.")
            return None

        # 3. 손금 탐지 (YOLOv8)
        results = self.model.predict(ai_input, conf=0.05, verbose=False)[0]

        analysis_report = {}
        detected_lines = {}

        if results.keypoints is not None:
            # 클래스별로 가장 신뢰도 높은 선 추출
            kpts = results.keypoints.xy.cpu().numpy()
            clses = results.boxes.cls.cpu().numpy()

            for idx, cls_id in enumerate(clses):
                line_name = self.LINE_MAP.get(int(cls_id))
                if not line_name: continue

                # 좌표 정제 (0,0 제외)
                points = np.array([(int(x), int(y)) for x, y in kpts[idx] if x > 0])

                # 특징 추출
                feats = self.extract_geometric_features(points, metrics['scale'])
                if feats:
                    detected_lines[line_name] = feats

        # 4. 관상학적 해석 수행
        analysis_report['Life Line'] = self.analyze_life_line(detected_lines.get('life'), metrics['mounts'], metrics['scale'])
        analysis_report['Head Line'] = self.analyze_head_line(detected_lines.get('head'), metrics['mounts'])
        analysis_report['Heart Line'] = self.analyze_heart_line(detected_lines.get('heart'), metrics['mounts'])
        analysis_report['Fate Line'] = self.analyze_fate_line(detected_lines.get('fate'), metrics['mounts'])

        return img, detected_lines, analysis_report

# --- 사용 예시 ---
if __name__ == "__main__":
    # 경로 설정
    BASE_PATH = '/content/drive/MyDrive/final_pro/mobileNet'
    model_path = os.path.join(BASE_PATH, 'num_02.pt')

    # 1. 인스턴스 생성
    analyzer = PalmPhysiognomyAnalyzer(model_path)

    # 2. 이미지 업로드 및 분석
    print("👇 분석할 손 사진을 업로드하세요.")
    uploaded = files.upload()

    if uploaded:
        img_file = list(uploaded.keys())[0]
        img_viz, lines, report = analyzer.run(img_file)

        # 3. 결과 출력 (Console)
        print(f"\n=== ✋ [{img_file}] 손금 분석 결과 ===")
        for line, result in report.items():
            print(f"\n[{line}]")
            for k, v in result.items():
                print(f"  - {k}: {v}")

        # 4. 시각화 (Simple Overlay)
        plt.figure(figsize=(10, 10))
        # 원본 이미지
        canvas = img_viz.copy()

        # 손금 그리기
        colors = {'life': (0, 255, 0), 'head': (255, 255, 0), 'heart': (255, 0, 0), 'fate': (255, 0, 255)}
        for name, data in lines.items():
            pts = data['points']
            # 선 그리기
            for i in range(len(pts)-1):
                cv2.line(canvas, tuple(pts[i]), tuple(pts[i+1]), colors.get(name, (255,255,255)), 8)
            # 이름표
            cv2.putText(canvas, name.upper(), tuple(pts[len(pts)//2]), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 4) # 테두리
            cv2.putText(canvas, name.upper(), tuple(pts[len(pts)//2]), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2) # 글씨

        plt.imshow(cv2.cvtColor(canvas, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title("AI Palmistry Result")
        plt.show()